In [ ]:
#Packages import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def volts(IVCurve): #input: csv file containing IV curve. Returns Vp, Vf, and ISat for initial guess
    rowskips = np.linspace(0, 60, 61, dtype=int)
    rowskips = np.append(rowskips, np.linspace(1063, 1070, 8, dtype=int)).tolist() #Make list of rows to skip
    IV = pd.read_csv(IVCurve, skiprows = rowskips)
    V = IV["SMU-1 Voltage (V)"].to_numpy()
    I = IV["SMU-1 Current (A)"].to_numpy()
    Isat = I[0]
    zero_crossings = np.where(np.diff(np.sign(I)))[0] #Locations where data crosses zero
    jump= 1
    for i in zero_crossings: #Data had weird jumps leading to multiple zeros. This for loop finds the smallest, where the true Vfloat should be
        if abs(I[i]-I[i+1])<jump:
            jump = abs(I[i]-I[i+1])
            Vfloat_index = i
    Vfloat = np.interp(0, I[Vfloat_index:Vfloat_index+2], V[Vfloat_index:Vfloat_index+2]) #Linear interpolation for zero crossing
    fit = np.polyfit(V, I, 8) #Fit to the data, returns polynomial coeffs
    Vfit = np.linspace (min(V), max(V))
    Ifit = np.poly1d(fit)(Vfit) #Vfit and Ifit are the points for the fit measured
    Iefit = Ifit-Isat
    d1Ifit = np.gradient(Iefit, Vfit) # first derivative, with both to test accuracy
    d1I = np.gradient(I, V)
    d2I = np.gradient(d1I, V)
    d2Ifit = np.gradient(d1Ifit, Vfit)
    Vfitfloat_index = int(np.argwhere(Vfit > Vfloat)[0]) #Before we determine point of inclecttion, we know it must be one greater than float voltage
    VPlasma_index=np.where(np.diff(np.sign(d2Ifit[Vfitfloat_index:])))[0] #where second derivative of the fit croses 0
    print(Vfloat, VPlasma_index)
    #Vplasma = np.interp(0, d2Ifit[Vfloat_index:Vfloat_index+2], Vfit[Vfloat_index:Vfloat_index+2]) #linear interp for Vp
    #Plotting code or troubleshooting. Copy, comment, and uncomment as needed
    """plt.figure(figsize=(10,4))
    plt.scatter(V, d2I, label='IV', color="red")
    plt.plot(Vfit, d2Ifit, label='Fit')
    plt.legend()
    plt.xlabel('V')
    plt.tight_layout()
    plt.xlim(5, 30)
    plt.ylim(-0.0025, 0.0025)
    plt.show()
    return Isat, Vfloat"""


In [ ]:
volts("LP/Trace_10p8mTorr_400W_1.csv")

3


C:\Users\Ahack\AppData\Local\Temp\ipykernel_3952\3635653937.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(int(np.argwhere(x >5)[0]))
